In [ ]:
import os
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
Train_folder="/content/drive/MyDrive/UCI HAR Dataset/train"
Test_folder="/content/drive/MyDrive/UCI HAR Dataset/test"

In [ ]:
activity_label={0:"WALKING", 1:"WALKING_UPSTAIRS", 2:"WALKING_DOWNSTAIRS", 3:"SITTING", 4:"STANDING", 5:"LAYING"}

In [ ]:
def read_X_y(X_path,y_path):
  X_dataset=[]
  y_dataset=[]
  with open(X_path,'r') as f:
    X_dataset=f.readlines()
  X_dataset=[[float(data) for data in x_data.strip().split()] for x_data in X_dataset]
  with open(y_path,'r') as f:
    y_dataset=f.readlines()
    y_dataset=[int(data.strip())-1 for data in y_dataset]
  return X_dataset, y_dataset

In [ ]:
x_path=os.path.join(Train_folder,"X_train.txt")
y_path=os.path.join(Train_folder,"y_train.txt")
X,y=read_X_y(x_path,y_path)

In [ ]:
x_path=os.path.join(Test_folder,"X_test.txt")
y_path=os.path.join(Test_folder,"y_test.txt")
X_t,y_t=read_X_y(x_path,y_path)

#Genetic algorithm to get best cnn model

 **1D CNN with tournament based approach**

In [ ]:
import numpy as np
import pandas as pd
import random
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder

# Load the dataset
X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None)

X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None)

# Reshape data
X_train_reshaped = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# One-hot encode labels
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Define hyperparameter space
n_filters_options = [32, 64, 128]
kernel_sizes_options = [3, 5, 7]
pool_sizes_options = [2, 3, 5]
activation_functions = ['relu', 'sigmoid', 'elu']

def create_cnn_model(num_filters, kernel_size, pool_size, activation_function):
    model = Sequential([
        Input(shape=(X_train_reshaped.shape[1], 1)),
        Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function),
        MaxPooling1D(pool_size=pool_size),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(y_train_encoded.shape[1], activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

def tournament_selection(population_scores, tournament_size=3):
    tournament = random.sample(population_scores, tournament_size)
    tournament.sort(key=lambda x: x[1], reverse=True)
    return tournament[0][0]

def genetic_algorithm(generations=4, population_size=8, tournament_size=3):
    population = [{
        'num_filters': random.choice(n_filters_options),
        'kernel_size': random.choice(kernel_sizes_options),
        'pool_size': random.choice(pool_sizes_options),
        'activation_function': random.choice(activation_functions)
    } for _ in range(population_size)]

    overall_best_model = None
    overall_best_accuracy = 0

    for generation in range(generations):
        print(f"\nGeneration {generation+1}")

        scores = []
        for individual in population:
            model = create_cnn_model(**individual)
            accuracy = evaluate_model(model, X_train_reshaped, y_train_encoded, X_test_reshaped, y_test_encoded)
            scores.append((individual, accuracy))
            print(f"Evaluated Model: {individual}, Accuracy: {accuracy:.4f}")

            if accuracy > overall_best_accuracy:
                overall_best_model = individual
                overall_best_accuracy = accuracy

        # Tournament selection for the next generation
        winners = [tournament_selection(scores, tournament_size) for _ in range(population_size)]
        print("Winners of this Generation:", winners)

        # Prepare for crossover and mutation
        new_population = []
        for i in range(0, len(winners), 2):
            if i+1 >= len(winners):  # Check if the population size is odd
                new_population.append(winners[i])
                break
            parent1, parent2 = winners[i], winners[i+1]

            # Assuming a 50% chance for crossover
            if random.random() < 0.5:
                parent1['kernel_size'], parent2['kernel_size'] = parent2['kernel_size'], parent1['kernel_size']
                print(f"Crossover happened: Kernel sizes swapped between two parents. New kernel sizes: {parent1['kernel_size']}, {parent2['kernel_size']}")

            # Mutation
            if random.random() < 0.2:
                old_value = parent1['num_filters']
                parent1['num_filters'] = random.choice(n_filters_options)
                print(f"Mutation on Parent1: Num_filters changed from {old_value} to {parent1['num_filters']}")

            new_population += [parent1, parent2]

        population = new_population

    print(f"\nOverall Best Model after Generation {generations}: {overall_best_model}, Accuracy: {overall_best_accuracy:.4f}")

# Ensure X_train_reshaped, y_train_encoded, X_test_reshaped, y_test_encoded are properly defined before calling this function
genetic_algorithm()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(



Generation 1
Evaluated Model: {'num_filters': 64, 'kernel_size': 3, 'pool_size': 3, 'activation_function': 'relu'}, Accuracy: 0.9572
Evaluated Model: {'num_filters': 128, 'kernel_size': 5, 'pool_size': 2, 'activation_function': 'elu'}, Accuracy: 0.9572
Evaluated Model: {'num_filters': 32, 'kernel_size': 3, 'pool_size': 5, 'activation_function': 'relu'}, Accuracy: 0.9552
Evaluated Model: {'num_filters': 128, 'kernel_size': 3, 'pool_size': 2, 'activation_function': 'relu'}, Accuracy: 0.9562
Evaluated Model: {'num_filters': 128, 'kernel_size': 7, 'pool_size': 5, 'activation_function': 'relu'}, Accuracy: 0.9481
Evaluated Model: {'num_filters': 64, 'kernel_size': 5, 'pool_size': 5, 'activation_function': 'elu'}, Accuracy: 0.9617
Evaluated Model: {'num_filters': 128, 'kernel_size': 3, 'pool_size': 5, 'activation_function': 'relu'}, Accuracy: 0.9464
Evaluated Model: {'num_filters': 64, 'kernel_size': 3, 'pool_size': 2, 'activation_function': 'sigmoid'}, Accuracy: 0.1822
Winners of this Gener

**After changing the CNN architecture (Adding Parallel layers)**

In [ ]:
import numpy as np
import pandas as pd
import random
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder

# Assuming the data loading and preprocessing steps are done
X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None)

X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None)

# Reshape data
X_train_reshaped = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# One-hot encode labels
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Hyperparameters space
n_filters_options = [32, 64, 128]
kernel_sizes_options = [3, 5, 7]
pool_sizes_options = [2, 3, 5]
activation_functions = ['relu', 'sigmoid', 'elu']

from keras.models import Model
from keras.layers import Concatenate, Average, Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam

def create_parallel_cnn_model(num_filters, kernel_size, pool_size, activation_function):

    input_shape = (X_train_reshaped.shape[1], 1)
    input_layer = Input(shape=input_shape)

    conv1a = Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function)(input_layer)
    pool1a = MaxPooling1D(pool_size=pool_size)(conv1a)

    conv1b = Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function)(input_layer)
    pool1b = MaxPooling1D(pool_size=pool_size)(conv1b)

    merged = Average()([pool1a, pool1b])


    conv2 = Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function)(merged)
    pool2 = MaxPooling1D(pool_size=pool_size)(conv2)

    flat = Flatten()(pool2)
    dense = Dense(128, activation='relu')(flat)
    dropout = Dropout(0.5)(dense)
    output_layer = Dense(y_train_encoded.shape[1], activation='softmax')(dropout)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model


def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

def tournament_selection(population_scores, tournament_size=3):
    tournament = random.sample(population_scores, tournament_size)
    tournament.sort(key=lambda x: x[1], reverse=True)
    return tournament[0][0]

def genetic_algorithm(generations=5, population_size=10, tournament_size=3):
    population = [{
        'num_filters': random.choice(n_filters_options),
        'kernel_size': random.choice(kernel_sizes_options),
        'pool_size': random.choice(pool_sizes_options),
        'activation_function': random.choice(activation_functions)
    } for _ in range(population_size)]

    overall_best_model = None
    overall_best_accuracy = 0

    for generation in range(generations):
        print(f"\nGeneration {generation+1}")

        scores = []
        for individual in population:
            model = create_parallel_cnn_model(**individual)
            accuracy = evaluate_model(model, X_train_reshaped, y_train_encoded, X_test_reshaped, y_test_encoded)
            scores.append((individual, accuracy))
            print(f"Evaluated Model: {individual}, Accuracy: {accuracy:.4f}")

            if accuracy > overall_best_accuracy:
                overall_best_model = individual
                overall_best_accuracy = accuracy


        winners = []
        for _ in range(population_size):
            winner = tournament_selection(scores, tournament_size)
            winners.append(winner)


        new_population = []
        for _ in range(0, population_size, 2):
            parent1, parent2 = random.sample(winners, 2)
            child1, child2 = parent1.copy(), parent2.copy()

            if random.random() < 0.5:
                child1['kernel_size'], child2['kernel_size'] = child2['kernel_size'], child1['kernel_size']
                print(f"Crossover happened: {child1['kernel_size']} swapped with {child2['kernel_size']}")

            # Mutation
            if random.random() < 0.2:
                child1['num_filters'] = random.choice(n_filters_options)
                print("Mutation: child1 num_filters changed")
            if random.random() < 0.2:
                child2['pool_size'] = random.choice(pool_sizes_options)
                print("Mutation: child2 pool_size changed")

            new_population.extend([child1, child2])

        population = new_population[:population_size]

    print(f"\nOverall Best Model: {overall_best_model}, Accuracy: {overall_best_accuracy:.4f}")

genetic_algorithm()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(



Generation 1
Evaluated Model: {'num_filters': 128, 'kernel_size': 7, 'pool_size': 3, 'activation_function': 'sigmoid'}, Accuracy: 0.3458
Evaluated Model: {'num_filters': 128, 'kernel_size': 7, 'pool_size': 3, 'activation_function': 'sigmoid'}, Accuracy: 0.1822
Evaluated Model: {'num_filters': 128, 'kernel_size': 7, 'pool_size': 2, 'activation_function': 'relu'}, Accuracy: 0.9447
Evaluated Model: {'num_filters': 128, 'kernel_size': 3, 'pool_size': 2, 'activation_function': 'relu'}, Accuracy: 0.9586
Evaluated Model: {'num_filters': 64, 'kernel_size': 5, 'pool_size': 5, 'activation_function': 'elu'}, Accuracy: 0.9457
Evaluated Model: {'num_filters': 64, 'kernel_size': 5, 'pool_size': 2, 'activation_function': 'relu'}, Accuracy: 0.9399
Evaluated Model: {'num_filters': 128, 'kernel_size': 5, 'pool_size': 5, 'activation_function': 'relu'}, Accuracy: 0.9389
Evaluated Model: {'num_filters': 32, 'kernel_size': 5, 'pool_size': 3, 'activation_function': 'relu'}, Accuracy: 0.9518
Evaluated Model:

**Adding the Additional parameters**

In [ ]:
import numpy as np
import pandas as pd
import random
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop
from tensorflow.keras.losses import categorical_crossentropy, mean_squared_error, mean_absolute_error

from sklearn.preprocessing import OneHotEncoder

# Assuming the data loading and preprocessing steps are done
X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None)

X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None)

# Reshape data
X_train_reshaped = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# One-hot encode labels
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Hyperparameters space
n_filters_options = [32, 64, 128]
kernel_sizes_options = [3, 5, 7]
pool_sizes_options = [2, 3, 5]
activation_functions = ['relu', 'sigmoid', 'elu']
next_functions = ['relu', 'sigmoid', 'elu']
units = [64, 128, 256]
optimizers = [Adam, Adagrad, RMSprop]
dense_losses = [categorical_crossentropy, mean_squared_error, mean_absolute_error]

from keras.models import Model
from keras.layers import Concatenate, Average, Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam

def create_parallel_cnn_model(num_filters, kernel_size, pool_size, activation_function, next_function, unit, optimizer, losses):
    input_shape = (X_train_reshaped.shape[1], 1)
    input_layer = Input(shape=input_shape)

    conv1a = Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function)(input_layer)
    pool1a = MaxPooling1D(pool_size=pool_size)(conv1a)

    conv1b = Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function)(input_layer)
    pool1b = MaxPooling1D(pool_size=pool_size)(conv1b)

    merged = Average()([pool1a, pool1b])

    conv2 = Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function)(merged)
    pool2 = MaxPooling1D(pool_size=pool_size)(conv2)

    flat = Flatten()(pool2)
    dense = Dense(units=unit, activation=next_function)(flat)
    dropout = Dropout(0.5)(dense)
    output_layer = Dense(y_train_encoded.shape[1], activation='softmax')(dropout)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=optimizer(learning_rate=0.001), loss=losses, metrics=['accuracy'])

    return model

def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

def tournament_selection(population_scores, tournament_size=3):
    tournament = random.sample(population_scores, tournament_size)
    tournament.sort(key=lambda x: x[1], reverse=True)
    return tournament[0][0]

def genetic_algorithm(generations=5, population_size=10, tournament_size=3):
    population = [{
        'num_filters': random.choice(n_filters_options),
        'kernel_size': random.choice(kernel_sizes_options),
        'pool_size': random.choice(pool_sizes_options),
        'activation_function': random.choice(activation_functions),
        'next_function': random.choice(next_functions),
        'unit': random.choice(units),
        'optimizer': random.choice(optimizers),
        'losses': random.choice(dense_losses),
    } for _ in range(population_size)]

    overall_best_model = None
    overall_best_accuracy = 0

    for generation in range(generations):
        print(f"\nGeneration {generation+1}")

        scores = []
        for individual in population:
            model = create_parallel_cnn_model(**individual)
            accuracy = evaluate_model(model, X_train_reshaped, y_train_encoded, X_test_reshaped, y_test_encoded)
            scores.append((individual, accuracy))
            print(f"Evaluated Model: {individual}, Accuracy: {accuracy:.4f}")

            if accuracy > overall_best_accuracy:
                overall_best_model = individual
                overall_best_accuracy = accuracy

        winners = []
        for _ in range(population_size):
            winner = tournament_selection(scores, tournament_size)
            winners.append(winner)

        new_population = []
        for _ in range(0, population_size, 2):
            parent1, parent2 = random.sample(winners, 2)
            child1, child2 = parent1.copy(), parent2.copy()

            if random.random() < 0.5:
                child1['kernel_size'], child2['kernel_size'] = child2['kernel_size'], child1['kernel_size']
                print(f"Crossover happened: {child1['kernel_size']} swapped with {child2['kernel_size']}")

            # Mutation
            if random.random() < 0.1:
                child1['num_filters'] = random.choice(n_filters_options)
                print("Mutation: child1 num_filters changed")
            if random.random() < 0.1:
                child2['pool_size'] = random.choice(pool_sizes_options)
                print("Mutation: child2 pool_size changed")

            new_population.extend([child1, child2])

        population = new_population[:population_size]

    print(f"\nOverall Best Model: {overall_best_model}, Accuracy: {overall_best_accuracy:.4f}")

genetic_algorithm()


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(



Generation 1
Evaluated Model: {'num_filters': 64, 'kernel_size': 3, 'pool_size': 5, 'activation_function': 'elu', 'next_function': 'elu', 'unit': 256, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'losses': <function mean_squared_error at 0x7d88964ff5b0>}, Accuracy: 0.9379
Evaluated Model: {'num_filters': 128, 'kernel_size': 7, 'pool_size': 2, 'activation_function': 'sigmoid', 'next_function': 'relu', 'unit': 256, 'optimizer': <class 'keras.src.optimizers.adagrad.Adagrad'>, 'losses': <function categorical_crossentropy at 0x7d88965288b0>}, Accuracy: 0.1805
Evaluated Model: {'num_filters': 128, 'kernel_size': 3, 'pool_size': 3, 'activation_function': 'relu', 'next_function': 'elu', 'unit': 256, 'optimizer': <class 'keras.src.optimizers.rmsprop.RMSprop'>, 'losses': <function mean_squared_error at 0x7d88964ff5b0>}, Accuracy: 0.9464
Evaluated Model: {'num_filters': 128, 'kernel_size': 3, 'pool_size': 5, 'activation_function': 'relu', 'next_function': 'sigmoid', 'unit': 256, 'optim

**Nueral Architecture Search 1D-CNN (Final)**

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Constants
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.7
POPULATION_SIZE = 15
GENERATIONS = 3
TOURNAMENT_SIZE = 3
INPUT_SHAPE = (561, 1)  # Example input shape, assuming 561 features
NUM_CLASSES = 6  # Adjust based on your dataset

n_filters_options = [16, 32, 64]
kernel_sizes_options = [3, 5]
pool_sizes_options = [2, 3]
activation_functions = ['relu', 'sigmoid', 'elu']
units = [64, 128, 256]
optimizers = ['adam', 'sgd']
dense_losses = ['sparse_categorical_crossentropy', 'categorical_crossentropy']
rates = [0.001, 0.01, 0.1]
batch_sizes = [16, 32, 64]
epochs = [10, 20, 30]

search_space = {
    'num_conv_layers': [1, 2, 3],
    'filters': n_filters_options,
    'kernel_size': kernel_sizes_options,
    'pool_size': pool_sizes_options,
    'activation': activation_functions,
    'learning_rate': rates,
    'dense_units': units,
    'batch_size': batch_sizes,
    'n_epoch': epochs,
}

def create_individual(search_space):
    individual = {
        'num_conv_layers_branch': random.choice(search_space['num_conv_layers']),
        'num_conv_layers_third': random.choice(search_space['num_conv_layers']),
        'conv_layers_branch': [],
        'conv_layers_third': [],
        'learning_rate': random.choice(search_space['learning_rate']),
        'dense_units': random.choice(search_space['dense_units']),
        'batch_size': random.choice(search_space['batch_size']),
        'n_epoch': random.choice(search_space['n_epoch']),
    }
    for _ in range(individual['num_conv_layers_branch']):
        individual['conv_layers_branch'].append({
            'filters': random.choice(search_space['filters']),
            'kernel_size': random.choice(search_space['kernel_size']),
            'pool_size': random.choice(search_space['pool_size']),
            'activation': random.choice(search_space['activation']),
        })
    for _ in range(individual['num_conv_layers_third']):
        individual['conv_layers_third'].append({
            'filters': random.choice(search_space['filters']),
            'kernel_size': random.choice(search_space['kernel_size']),
            'pool_size': random.choice(search_space['pool_size']),
            'activation': random.choice(search_space['activation']),
        })
    return individual

def create_model(individual):
    inputs = Input(shape=INPUT_SHAPE)
    branches = []
    for _ in range(2):
        x = inputs
        for layer_spec in individual['conv_layers_branch']:
            x = Conv1D(filters=layer_spec['filters'], kernel_size=layer_spec['kernel_size'],
                       activation=layer_spec['activation'], padding='same')(x)
            x = MaxPooling1D(pool_size=layer_spec['pool_size'])(x)
        branches.append(x)
    merged = concatenate(branches)
    x = merged
    for layer_spec in individual['conv_layers_third']:
        x = Conv1D(filters=layer_spec['filters'], kernel_size=layer_spec['kernel_size'],
                   activation=layer_spec['activation'], padding='same')(x)
        x = MaxPooling1D(pool_size=layer_spec['pool_size'])(x)
    x = Flatten()(x)
    x = Dense(units=individual['dense_units'], activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=individual['learning_rate']),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# (Continue with the other functions like evaluate_model, tournament_selection, crossover, mutate, and genetic_algorithm)
def evaluate_model(model, X_train, y_train, X_val, y_val, individual):
    history = model.fit(X_train, y_train, epochs=individual['n_epoch'], batch_size=individual['batch_size'], verbose=0, validation_data=(X_val, y_val))
    _, accuracy = model.evaluate(X_val, y_val, verbose=0)
    return accuracy

# Tournament Selection
def tournament_selection(population_scores, tournament_size):
    tournament = random.sample(population_scores, min(tournament_size, len(population_scores)))
    tournament.sort(key=lambda x: x[1], reverse=True)
    return tournament[0][0]

# Crossover operation
def crossover(individual1, individual2):
    print("\nPerforming crossover between two individuals.")
    print(f"Parent 1 configuration before crossover: {individual1}")
    print(f"Parent 2 configuration before crossover: {individual2}")
    child1, child2 = individual1.copy(), individual2.copy()
    if random.random() < CROSSOVER_RATE:
        # Swap branch configurations
        child1['conv_layers_branch'], child2['conv_layers_branch'] = child2['conv_layers_branch'], child1['conv_layers_branch']
        # Swap third layer configurations
        child1['conv_layers_third'], child2['conv_layers_third'] = child2['conv_layers_third'], child1['conv_layers_third']
        print("Crossover result:")
        print(f"Child 1 configuration: {child1}")
        print(f"Child 2 configuration: {child2}")
    else:
        print("No crossover occurred.")
    return child1, child2

# Enhanced mutation operation to print detailed information
def mutate(individual):
    print("\nChecking for mutation.")
    original_configuration = individual.copy()
    mutation_occurred = False
    if random.random() < MUTATION_RATE:
        print("Mutation occurred on num_conv_layers_branch.")
        individual['num_conv_layers_branch'] = random.choice(search_space['num_conv_layers'])
        mutation_occurred = True
    if random.random() < MUTATION_RATE:
        print("Mutation occurred on num_conv_layers_third.")
        individual['num_conv_layers_third'] = random.choice(search_space['num_conv_layers'])
        mutation_occurred = True
    if mutation_occurred:
        print(f"Original configuration: {original_configuration}")
        print(f"Mutated configuration: {individual}")
    else:
        print("No mutation occurred.")
    return individual


def genetic_algorithm(X_train, y_train, X_val, y_val, generations=GENERATIONS, population_size=POPULATION_SIZE, tournament_size=TOURNAMENT_SIZE):
    population = [create_individual(search_space) for _ in range(population_size)]
    best_individual = None
    best_accuracy = -1

    for generation in range(generations):
        print(f'\n--- Generation {generation + 1} ---')
        population_scores = []
        for index, individual in enumerate(population):
            model = create_model(individual)
            accuracy = evaluate_model(model, X_train, y_train, X_val, y_val, individual)
            population_scores.append((individual, accuracy))

            # Print detailed individual configuration along with accuracy
            individual_details = ', '.join([f"{key}: {value}" for key, value in individual.items()])
            print(f'Individual {index + 1}: Accuracy = {accuracy:.4f}, Configuration: {{{individual_details}}}')

            if accuracy > best_accuracy:
                best_individual = individual
                best_accuracy = accuracy

        print("\nTournament Selection:")
        selected_individuals = []
        for _ in range(population_size):
            selected = tournament_selection(population_scores, tournament_size)
            selected_individuals.append(selected)
            selected_details = ', '.join([f"{key}: {value}" for key, value in selected.items()])
            print(f'Selected configuration: {{{selected_details}}}')

        print("\nCrossover and Mutation Phase:")
        next_population = []
        for i in range(0, population_size, 2):
            parent1, parent2 = selected_individuals[i], selected_individuals[min(i + 1, population_size - 1)]
            child1, child2 = crossover(parent1, parent2)
            next_population.append(mutate(child1))
            next_population.append(mutate(child2))

        population = next_population
        print(f'\nEnd of Generation {generation + 1}, Best Accuracy: {best_accuracy:.4f}')

    return best_individual, best_accuracy
if __name__ == "__main__":
    # Load and preprocess data
    X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
    y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None)
    X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
    y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None)

    # Reshape and encode data
    X_train_reshaped = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))
    encoder = OneHotEncoder(sparse=False)
    y_train_encoded = encoder.fit_transform(y_train)
    y_test_encoded = encoder.transform(y_test)
    X_train_reshaped, X_val_reshaped, y_train_encoded, y_val_encoded = train_test_split(
        X_train_reshaped, y_train_encoded, test_size=0.2, random_state=42
    )

    # Run the genetic algorithm
    best_individual, best_accuracy = genetic_algorithm(
        X_train_reshaped, y_train_encoded, X_val_reshaped, y_val_encoded
    )
    print("\nBest Individual Configuration and Accuracy:")
    individual_details = ', '.join([f"{key}: {value}" for key, value in best_individual.items()])
    print(f"Configuration: {{{individual_details}}}")
    print(f"Accuracy: {best_accuracy:.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(



--- Generation 1 ---
Individual 1: Accuracy = 0.4052, Configuration: {num_conv_layers_branch: 1, num_conv_layers_third: 3, conv_layers_branch: [{'filters': 32, 'kernel_size': 5, 'pool_size': 3, 'activation': 'relu'}], conv_layers_third: [{'filters': 16, 'kernel_size': 5, 'pool_size': 2, 'activation': 'sigmoid'}, {'filters': 16, 'kernel_size': 5, 'pool_size': 3, 'activation': 'sigmoid'}, {'filters': 32, 'kernel_size': 3, 'pool_size': 2, 'activation': 'sigmoid'}], learning_rate: 0.001, dense_units: 128, batch_size: 64, n_epoch: 10}
Individual 2: Accuracy = 0.1903, Configuration: {num_conv_layers_branch: 2, num_conv_layers_third: 3, conv_layers_branch: [{'filters': 32, 'kernel_size': 3, 'pool_size': 2, 'activation': 'relu'}, {'filters': 16, 'kernel_size': 3, 'pool_size': 3, 'activation': 'sigmoid'}], conv_layers_third: [{'filters': 64, 'kernel_size': 3, 'pool_size': 3, 'activation': 'sigmoid'}, {'filters': 64, 'kernel_size': 3, 'pool_size': 2, 'activation': 'sigmoid'}, {'filters': 32, 'k

**EXPERIMENTS ON NAS**

In [ ]:
`
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Constants
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.7
POPULATION_SIZE = 15
GENERATIONS = 3
TOURNAMENT_SIZE = 3
INPUT_SHAPE = (561, 1)  # Example input shape, assuming 561 features
NUM_CLASSES = 6  # Adjust based on your dataset

n_filters_options = [16, 32, 64]
kernel_sizes_options = [3, 5]
pool_sizes_options = [2,3]
activation_functions = ['relu', 'sigmoid','elu']
next_functions = ['flatten', 'dense']
units = [64, 128, 256]
optimizers = ['adam', 'sgd']
dense_losses = ['sparse_categorical_crossentropy', 'categorical_crossentropy']
rates = [0.001, 0.01, 0.1]
batch_sizes = [16, 32, 64]
epochs = [10, 20, 30]

search_space = {
    'num_conv_layers': [1, 2, 3],
    'filters': n_filters_options,
    'kernel_size': kernel_sizes_options,
    'pool_size': pool_sizes_options,
    'activation': activation_functions,
    'learning_rate': rates,
    'dense_units': units,
    'batch_size': batch_sizes,
    'n_epoch': epochs,
}

# Function to create an individual (CNN architecture configuration)
def create_individual(search_space):
    individual = {
        'num_conv_layers_branch': random.choice(search_space['num_conv_layers']),
        'num_conv_layers_third': random.choice(search_space['num_conv_layers']),
        'conv_layers_branch': [],
        'conv_layers_third': [],
        'learning_rate': random.choice(search_space['learning_rate']),
        'dense_units': random.choice(search_space['dense_units']),
        'batch_size': random.choice(search_space['batch_size']),
        'n_epoch': random.choice(search_space['n_epoch']),
    }
    for _ in range(individual['num_conv_layers_branch']):
        individual['conv_layers_branch'].append({
            'filters': random.choice(search_space['filters']),
            'kernel_size': random.choice(search_space['kernel_size']),
            'activation': random.choice(search_space['activation']),
        })
    for _ in range(individual['num_conv_layers_third']):
        individual['conv_layers_third'].append({
            'filters': random.choice(search_space['filters']),
            'kernel_size': random.choice(search_space['kernel_size']),
            'activation': random.choice(search_space['activation']),
        })
    return individual

# Function to create a CNN model based on an individual's specifications
def create_model(individual):
    inputs = Input(shape=INPUT_SHAPE)
    branches = []
    for _ in range(2):  # Two parallel branches
        x = inputs
        for layer_spec in individual['conv_layers_branch']:
            x = Conv1D(filters=layer_spec['filters'], kernel_size=layer_spec['kernel_size'],
                       activation=layer_spec['activation'], padding='same')(x)
            x = MaxPooling1D(pool_size=search_space['pool_size'][0])(x)
        branches.append(x)
    merged = concatenate(branches)
    x = merged
    for layer_spec in individual['conv_layers_third']:
        x = Conv1D(filters=layer_spec['filters'], kernel_size=layer_spec['kernel_size'],
                   activation=layer_spec['activation'], padding='same')(x)
        x = MaxPooling1D(pool_size=search_space['pool_size'][0])(x)
    x = Flatten()(x)
    x = Dense(units=individual['dense_units'], activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=individual['learning_rate']),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to evaluate the model
def evaluate_model(model, X_train, y_train, X_val, y_val, individual):
    history = model.fit(X_train, y_train, epochs=individual['n_epoch'], batch_size=individual['batch_size'], verbose=0, validation_data=(X_val, y_val))
    _, accuracy = model.evaluate(X_val, y_val, verbose=0)
    return accuracy

# Tournament Selection
def tournament_selection(population_scores, tournament_size):
    tournament = random.sample(population_scores, min(tournament_size, len(population_scores)))
    tournament.sort(key=lambda x: x[1], reverse=True)
    return tournament[0][0]

# Crossover operation
def crossover(individual1, individual2):
    print("\nPerforming crossover between two individuals.")
    print(f"Parent 1 configuration before crossover: {individual1}")
    print(f"Parent 2 configuration before crossover: {individual2}")
    child1, child2 = individual1.copy(), individual2.copy()
    if random.random() < CROSSOVER_RATE:
        # Swap branch configurations
        child1['conv_layers_branch'], child2['conv_layers_branch'] = child2['conv_layers_branch'], child1['conv_layers_branch']
        # Swap third layer configurations
        child1['conv_layers_third'], child2['conv_layers_third'] = child2['conv_layers_third'], child1['conv_layers_third']
        print("Crossover result:")
        print(f"Child 1 configuration: {child1}")
        print(f"Child 2 configuration: {child2}")
    else:
        print("No crossover occurred.")
    return child1, child2

# Enhanced mutation operation to print detailed information
def mutate(individual):
    print("\nChecking for mutation.")
    original_configuration = individual.copy()
    mutation_occurred = False
    if random.random() < MUTATION_RATE:
        print("Mutation occurred on num_conv_layers_branch.")
        individual['num_conv_layers_branch'] = random.choice(search_space['num_conv_layers'])
        mutation_occurred = True
    if random.random() < MUTATION_RATE:
        print("Mutation occurred on num_conv_layers_third.")
        individual['num_conv_layers_third'] = random.choice(search_space['num_conv_layers'])
        mutation_occurred = True
    if mutation_occurred:
        print(f"Original configuration: {original_configuration}")
        print(f"Mutated configuration: {individual}")
    else:
        print("No mutation occurred.")
    return individual


def genetic_algorithm(X_train, y_train, X_val, y_val, generations=GENERATIONS, population_size=POPULATION_SIZE, tournament_size=TOURNAMENT_SIZE):
    population = [create_individual(search_space) for _ in range(population_size)]
    best_individual = None
    best_accuracy = -1

    for generation in range(generations):
        print(f'\n--- Generation {generation + 1} ---')
        population_scores = []
        for index, individual in enumerate(population):
            model = create_model(individual)
            accuracy = evaluate_model(model, X_train, y_train, X_val, y_val, individual)
            population_scores.append((individual, accuracy))

            # Print detailed individual configuration along with accuracy
            individual_details = ', '.join([f"{key}: {value}" for key, value in individual.items()])
            print(f'Individual {index + 1}: Accuracy = {accuracy:.4f}, Configuration: {{{individual_details}}}')

            if accuracy > best_accuracy:
                best_individual = individual
                best_accuracy = accuracy

        print("\nTournament Selection:")
        selected_individuals = []
        for _ in range(population_size):
            selected = tournament_selection(population_scores, tournament_size)
            selected_individuals.append(selected)
            selected_details = ', '.join([f"{key}: {value}" for key, value in selected.items()])
            print(f'Selected configuration: {{{selected_details}}}')

        print("\nCrossover and Mutation Phase:")
        next_population = []
        for i in range(0, population_size, 2):
            parent1, parent2 = selected_individuals[i], selected_individuals[min(i + 1, population_size - 1)]
            child1, child2 = crossover(parent1, parent2)
            next_population.append(mutate(child1))
            next_population.append(mutate(child2))

        population = next_population
        print(f'\nEnd of Generation {generation + 1}, Best Accuracy: {best_accuracy:.4f}')

    return best_individual, best_accuracy

# Ensure the rest of your script is unchanged, and the crossover, mutate, and model creation functions are correctly defined as earlier.

# Main execution block remains the same
if __name__ == "__main__":
    # Load and preprocess data
    X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
    y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None)
    X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
    y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None)

    # Reshape data
    X_train_reshaped = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

    # One-hot encode labels
    encoder = OneHotEncoder(sparse=False)
    y_train_encoded = encoder.fit_transform(y_train)
    y_test_encoded = encoder.transform(y_test)

    # Split the dataset into train and validation sets
    X_train_reshaped, X_val_reshaped, y_train_encoded, y_val_encoded = train_test_split(
        X_train_reshaped, y_train_encoded, test_size=0.2, random_state=42
    )

    # Run the genetic algorithm
    best_individual, best_accuracy = genetic_algorithm(
        X_train_reshaped, y_train_encoded, X_val_reshaped, y_val_encoded
    )
    print("\nBest Individual Configuration and Accuracy:")
    individual_details = ', '.join([f"{key}: {value}" for key, value in best_individual.items()])
    print(f"Configuration: {{{individual_details}}}")
    print(f"Accuracy: {best_accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(



--- Generation 1 ---
Individual 1: Accuracy = 0.1903, Configuration: {num_conv_layers_branch: 1, num_conv_layers_third: 1, conv_layers_branch: [{'filters': 64, 'kernel_size': 5, 'activation': 'sigmoid'}], conv_layers_third: [{'filters': 32, 'kernel_size': 5, 'activation': 'sigmoid'}], learning_rate: 0.001, dense_units: 64, batch_size: 16, n_epoch: 20}
Individual 2: Accuracy = 0.1903, Configuration: {num_conv_layers_branch: 3, num_conv_layers_third: 1, conv_layers_branch: [{'filters': 64, 'kernel_size': 3, 'activation': 'elu'}, {'filters': 16, 'kernel_size': 5, 'activation': 'sigmoid'}, {'filters': 64, 'kernel_size': 3, 'activation': 'elu'}], conv_layers_third: [{'filters': 16, 'kernel_size': 3, 'activation': 'relu'}], learning_rate: 0.1, dense_units: 256, batch_size: 16, n_epoch: 30}
Individual 3: Accuracy = 0.1876, Configuration: {num_conv_layers_branch: 2, num_conv_layers_third: 3, conv_layers_branch: [{'filters': 16, 'kernel_size': 3, 'activation': 'elu'}, {'filters': 32, 'kernel_s

**EXPERIMENT**

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Constants
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.7
POPULATION_SIZE = 15
GENERATIONS = 3
TOURNAMENT_SIZE = 3
INPUT_SHAPE = (561, 1)  # Example input shape, assuming 561 features
NUM_CLASSES = 6  # Adjust based on your dataset

n_filters_options = [16, 32, 64]
kernel_sizes_options = [3, 5]
pool_sizes_options = [2,3]
activation_functions = ['relu', 'sigmoid','elu']
next_functions = ['flatten', 'dense']
units = [64, 128, 256]
optimizers = ['adam', 'sgd']
dense_losses = ['sparse_categorical_crossentropy', 'categorical_crossentropy']
rates = [0.001, 0.01, 0.1]
batch_sizes = [16, 32, 64]
epochs = [10, 20, 30]

search_space = {
    'num_conv_layers': [1, 2, 3],
    'filters': n_filters_options,
    'kernel_size': kernel_sizes_options,
    'pool_size': pool_sizes_options,
    'activation': activation_functions,
    'learning_rate': rates,
    'dense_units': units,
    'batch_size': batch_sizes,
    'n_epoch': epochs,
}

# Function to create an individual (CNN architecture configuration)
def create_individual(search_space):
    individual = {
        'num_conv_layers_branch': random.choice(search_space['num_conv_layers']),
        'num_conv_layers_third': random.choice(search_space['num_conv_layers']),
        'conv_layers_branch': [],
        'conv_layers_third': [],
        'learning_rate': random.choice(search_space['learning_rate']),
        'dense_units': random.choice(search_space['dense_units']),
        'batch_size': random.choice(search_space['batch_size']),
        'n_epoch': random.choice(search_space['n_epoch']),
    }
    for _ in range(individual['num_conv_layers_branch']):
        individual['conv_layers_branch'].append({
            'filters': random.choice(search_space['filters']),
            'kernel_size': random.choice(search_space['kernel_size']),
            'activation': random.choice(search_space['activation']),
        })
    for _ in range(individual['num_conv_layers_third']):
        individual['conv_layers_third'].append({
            'filters': random.choice(search_space['filters']),
            'kernel_size': random.choice(search_space['kernel_size']),
            'activation': random.choice(search_space['activation']),
        })
    return individual

# Function to create a CNN model based on an individual's specifications
def create_model(individual):
    inputs = Input(shape=INPUT_SHAPE)
    branches = []
    for _ in range(2):  # Two parallel branches
        x = inputs
        for layer_spec in individual['conv_layers_branch']:
            x = Conv1D(filters=layer_spec['filters'], kernel_size=layer_spec['kernel_size'],
                       activation=layer_spec['activation'], padding='same')(x)
            x = MaxPooling1D(pool_size=search_space['pool_size'][0])(x)
        branches.append(x)
    merged = concatenate(branches)
    x = merged
    for layer_spec in individual['conv_layers_third']:
        x = Conv1D(filters=layer_spec['filters'], kernel_size=layer_spec['kernel_size'],
                   activation=layer_spec['activation'], padding='same')(x)
        x = MaxPooling1D(pool_size=search_space['pool_size'][0])(x)
    x = Flatten()(x)
    x = Dense(units=individual['dense_units'], activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=individual['learning_rate']),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to evaluate the model
def evaluate_model(model, X_train, y_train, X_val, y_val, individual):
    history = model.fit(X_train, y_train, epochs=individual['n_epoch'], batch_size=individual['batch_size'], verbose=0, validation_data=(X_val, y_val))
    _, accuracy = model.evaluate(X_val, y_val, verbose=0)
    return accuracy

# Tournament Selection
# Tournament Selection
def tournament_selection(population_scores, tournament_size):
    tournament = random.sample(population_scores, min(tournament_size, len(population_scores)))
    tournament_with_all_branches = [indiv for indiv in tournament if len(indiv[0]['conv_layers_branch']) == 2 and len(indiv[0]['conv_layers_third']) == 1]  # Adjust based on the number of branches
    if tournament_with_all_branches:
        tournament_with_all_branches.sort(key=lambda x: x[1], reverse=True)
        return tournament_with_all_branches[0][0]
    else:
        # If no individuals with all branches are found, select from the original tournament
        tournament.sort(key=lambda x: x[1], reverse=True)
        return tournament[0][0]


# Crossover operation
def crossover(individual1, individual2):
    print("\nPerforming crossover between two individuals.")
    print(f"Parent 1 configuration before crossover: {individual1}")
    print(f"Parent 2 configuration before crossover: {individual2}")
    child1, child2 = individual1.copy(), individual2.copy()
    if random.random() < CROSSOVER_RATE:
        # Swap branch configurations
        child1['conv_layers_branch'], child2['conv_layers_branch'] = child2['conv_layers_branch'], child1['conv_layers_branch']
        # Swap third layer configurations
        child1['conv_layers_third'], child2['conv_layers_third'] = child2['conv_layers_third'], child1['conv_layers_third']
        print("Crossover result:")
        print(f"Child 1 configuration: {child1}")
        print(f"Child 2 configuration: {child2}")
    else:
        print("No crossover occurred.")
    return child1, child2

# Enhanced mutation operation to print detailed information
def mutate(individual):
    print("\nChecking for mutation.")
    original_configuration = individual.copy()
    mutation_occurred = False
    if random.random() < MUTATION_RATE:
        print("Mutation occurred on num_conv_layers_branch.")
        individual['num_conv_layers_branch'] = random.choice(search_space['num_conv_layers'])
        mutation_occurred = True
    if random.random() < MUTATION_RATE:
        print("Mutation occurred on num_conv_layers_third.")
        individual['num_conv_layers_third'] = random.choice(search_space['num_conv_layers'])
        mutation_occurred = True
    if mutation_occurred:
        print(f"Original configuration: {original_configuration}")
        print(f"Mutated configuration: {individual}")
    else:
        print("No mutation occurred.")
    return individual


def genetic_algorithm(X_train, y_train, X_val, y_val, generations=GENERATIONS, population_size=POPULATION_SIZE, tournament_size=TOURNAMENT_SIZE):
    population = [create_individual(search_space) for _ in range(population_size)]
    best_individual = None
    best_accuracy = -1

    # Inside the genetic_algorithm function
    for generation in range(generations):
        print(f'\n--- Generation {generation + 1} ---')
        population_scores = []
        for index, individual in enumerate(population):
            model = create_model(individual)
            accuracy = evaluate_model(model, X_train, y_train, X_val, y_val, individual)
            population_scores.append((individual, accuracy))

            # Print detailed individual configuration along with accuracy
            individual_details = ', '.join([f"{key}: {value}" for key, value in individual.items()])
            print(f'Individual {index + 1}: Accuracy = {accuracy:.4f}, Configuration: {{{individual_details}}}')

            if accuracy > best_accuracy:
                best_individual = individual
                best_accuracy = accuracy

        print("\nTournament Selection:")
        selected_individuals = []
        for index, (individual, accuracy) in enumerate(population_scores):
            selected_individuals.append(individual)
            individual_details = ', '.join([f"{key}: {value}" for key, value in individual.items()])
            print(f'Selected configuration {index + 1}: Accuracy = {accuracy:.4f}, Configuration: {{{individual_details}}}')

        print("\nCrossover and Mutation Phase:")
        next_population = []
        for i in range(0, population_size, 2):
            parent1, parent2 = selected_individuals[i], selected_individuals[min(i + 1, population_size - 1)]
            child1, child2 = crossover(parent1, parent2)
            next_population.append(mutate(child1))
            next_population.append(mutate(child2))

        population = next_population
        print(f'\nEnd of Generation {generation + 1}, Best Accuracy: {best_accuracy:.4f}')
        print(f'\n--- Generation {generation + 1} ---')


    return best_individual, best_accuracy

# Ensure the rest of your script is unchanged, and the crossover, mutate, and model creation functions are correctly defined as earlier.

# Main execution block remains the same
if __name__ == "__main__":
    # Load and preprocess data
    X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
    y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None)
    X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
    y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None)

    # Reshape data
    X_train_reshaped = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

    # One-hot encode labels
    encoder = OneHotEncoder(sparse=False)
    y_train_encoded = encoder.fit_transform(y_train)
    y_test_encoded = encoder.transform(y_test)

    # Split the dataset into train and validation sets
    X_train_reshaped, X_val_reshaped, y_train_encoded, y_val_encoded = train_test_split(
        X_train_reshaped, y_train_encoded, test_size=0.2, random_state=42
    )

    # Run the genetic algorithm
    best_individual, best_accuracy = genetic_algorithm(
        X_train_reshaped, y_train_encoded, X_val_reshaped, y_val_encoded
    )
    print("Accuracy =", best_accuracy, "Configuration = ", best_individual)



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(



--- Generation 1 ---
Individual 1: Accuracy = 0.1903, Configuration: {num_conv_layers_branch: 2, num_conv_layers_third: 1, conv_layers_branch: [{'filters': 32, 'kernel_size': 5, 'activation': 'sigmoid'}, {'filters': 32, 'kernel_size': 3, 'activation': 'elu'}], conv_layers_third: [{'filters': 64, 'kernel_size': 3, 'activation': 'elu'}], learning_rate: 0.001, dense_units: 64, batch_size: 64, n_epoch: 20}
Individual 2: Accuracy = 0.9592, Configuration: {num_conv_layers_branch: 3, num_conv_layers_third: 1, conv_layers_branch: [{'filters': 32, 'kernel_size': 5, 'activation': 'relu'}, {'filters': 32, 'kernel_size': 3, 'activation': 'elu'}, {'filters': 32, 'kernel_size': 3, 'activation': 'sigmoid'}], conv_layers_third: [{'filters': 64, 'kernel_size': 5, 'activation': 'relu'}], learning_rate: 0.001, dense_units: 128, batch_size: 16, n_epoch: 10}
Individual 3: Accuracy = 0.1903, Configuration: {num_conv_layers_branch: 3, num_conv_layers_third: 3, conv_layers_branch: [{'filters': 32, 'kernel_si

In [ ]:
print("Accuracy =", best_accuracy, "Configuration = ", best_individual)

Accuracy = 0.9809653162956238 Configuration =  {'num_conv_layers_branch': 2, 'num_conv_layers_third': 2, 'conv_layers_branch': [{'filters': 64, 'kernel_size': 3, 'activation': 'relu'}, {'filters': 64, 'kernel_size': 5, 'activation': 'elu'}], 'conv_layers_third': [{'filters': 32, 'kernel_size': 5, 'activation': 'sigmoid'}, {'filters': 16, 'kernel_size': 3, 'activation': 'elu'}], 'learning_rate': 0.001, 'dense_units': 128, 'batch_size': 32, 'n_epoch': 10}


**Experiments on implememting GA on basic 1D-CNN Structures**

In [ ]:
import numpy as np
import pandas as pd
import random
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder
import time

# Load the dataset
X_train = pd.read_csv('/content/drive/MyDrive/UCI_HAR_Dataset/UCI/train/X_train.txt', delim_whitespace=True, header=None)
y_train = pd.read_csv('/content/drive/MyDrive/UCI_HAR_Dataset/UCI/train/y_train.txt', delim_whitespace=True, header=None)

X_test = pd.read_csv('/content/drive/MyDrive/UCI_HAR_Dataset/UCI/test/X_test.txt', delim_whitespace=True, header=None)
y_test = pd.read_csv('/content/drive/MyDrive/UCI_HAR_Dataset/UCI/test/y_test.txt', delim_whitespace=True, header=None)

# Reshape data
X_train_reshaped = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# One-hot encode labels
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Hyperparameters space
n_filters_options = [32, 64, 128]
kernel_sizes_options = [3, 5, 7]
pool_sizes_options = [2, 3, 5]
activation_functions = ['relu', 'sigmoid', 'elu']

def create_cnn_model(num_filters, kernel_size, pool_size, activation_function):
    model = Sequential([
        Input(shape=(X_train_reshaped.shape[1], 1)),
        Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function),
        MaxPooling1D(pool_size=pool_size),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(y_train_encoded.shape[1], activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

def genetic_algorithm(generations=5, population_size=10):
    population = [{
        'num_filters': random.choice(n_filters_options),
        'kernel_size': random.choice(kernel_sizes_options),
        'pool_size': random.choice(pool_sizes_options),
        'activation_function': random.choice(activation_functions)
    } for _ in range(population_size)]

    for generation in range(generations):
        print(f"Generation {generation+1}")

        # Evaluate the population
        scores = []
        for individual in population:
            model = create_cnn_model(**individual)
            accuracy = evaluate_model(model, X_train_reshaped, y_train_encoded, X_test_reshaped, y_test_encoded)
            scores.append((individual, accuracy))
            print(f"Model with {individual} achieved accuracy: {accuracy:.4f}")

        # Rank individuals
        scores.sort(key=lambda x: x[1], reverse=True)
        top_performers = scores[:population_size//2]

        # Create next generation
        next_generation = [x[0] for x in top_performers]
        while len(next_generation) < population_size:
            parent = random.choice(next_generation)
            child = parent.copy()
            # Mutation
            if random.random() < 0.1:
                child['num_filters'] = random.choice(n_filters_options)
            if random.random() < 0.1:
                child['kernel_size'] = random.choice(kernel_sizes_options)
            if random.random() < 0.1:
                child['pool_size'] = random.choice(pool_sizes_options)
            if random.random() < 0.1:
                child['activation_function'] = random.choice(activation_functions)
            next_generation.append(child)

        population = next_generation

    best_individual = scores[0][0]
    best_accuracy = scores[0][1]
    print(f"Best Model: {best_individual} with accuracy: {best_accuracy:.4f}")

genetic_algorithm()


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Generation 1
Model with {'num_filters': 128, 'kernel_size': 5, 'pool_size': 3, 'activation_function': 'relu'} achieved accuracy: 0.9640
Model with {'num_filters': 64, 'kernel_size': 5, 'pool_size': 2, 'activation_function': 'elu'} achieved accuracy: 0.9515
Model with {'num_filters': 32, 'kernel_size': 7, 'pool_size': 2, 'activation_function': 'sigmoid'} achieved accuracy: 0.1822
Model with {'num_filters': 32, 'kernel_size': 3, 'pool_size': 3, 'activation_function': 'elu'} achieved accuracy: 0.9555
Model with {'num_filters': 128, 'kernel_size': 7, 'pool_size': 2, 'activation_function': 'relu'} achieved accuracy: 0.9494
Model with {'num_filters': 128, 'kernel_size': 7, 'pool_size': 3, 'activation_function': 'relu'} achieved accuracy: 0.9467
Model with {'num_filters': 64, 'kernel_size': 3, 'pool_size': 3, 'activation_function': 'relu'} achieved accuracy: 0.9593
Model with {'num_filters': 32, 'kernel_size': 5, 'pool_size': 3, 'activation_function': 'sigmoid'} achieved accuracy: 0.1822
Mode

In [ ]:
import numpy as np
import pandas as pd
import random
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop
from tensorflow.keras.losses import categorical_crossentropy, mean_squared_error, mean_absolute_error


from sklearn.preprocessing import OneHotEncoder

# Assuming the data loading and preprocessing steps are done
X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None)

X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None)

# Reshape data
X_train_reshaped = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# One-hot encode labels
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Hyperparameters space
n_filters_options = [32, 64, 128]
kernel_sizes_options = [3, 5, 7]
pool_sizes_options = [2, 3, 5]
activation_functions = ['relu', 'sigmoid', 'elu']
next_functions = ['relu', 'sigmoid', 'elu']
units = [64,128,256]
optimizers = [Adam , Adagrad, RMSprop]
dense_losses = [categorical_crossentropy, mean_squared_error, mean_absolute_error]

from keras.models import Model
from keras.layers import Concatenate, Average, Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam

def create_parallel_cnn_model(num_filters, kernel_size, pool_size, activation_function,next_function, unit, optimizer,losses):

    input_shape = (X_train_reshaped.shape[1], 1)
    input_layer = Input(shape=input_shape)

    conv1a = Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function)(input_layer)
    pool1a = MaxPooling1D(pool_size=pool_size)(conv1a)

    conv1b = Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function)(input_layer)
    pool1b = MaxPooling1D(pool_size=pool_size)(conv1b)

    merged = Average()([pool1a, pool1b])


    conv2 = Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function)(merged)
    pool2 = MaxPooling1D(pool_size=pool_size)(conv2)

    flat = Flatten()(pool2)
    dense = Dense(units=unit, activation=next_function)(flat)
    dropout = Dropout(0.5)(dense)
    output_layer = Dense(y_train_encoded.shape[1], activation='softmax')(dropout)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=optimizer(learning_rate=0.001), loss=losses, metrics=['accuracy'])

    return model


def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

def tournament_selection(population_scores, tournament_size=3):
    tournament = random.sample(population_scores, tournament_size)
    tournament.sort(key=lambda x: x[1], reverse=True)
    return tournament[0][0]

def genetic_algorithm(generations=5, population_size=10, tournament_size=3):
    population = [{
        'num_filters': random.choice(n_filters_options),
        'kernel_size': random.choice(kernel_sizes_options),
        'pool_size': random.choice(pool_sizes_options),
        'activation_function': random.choice(activation_functions),
        'next_function': random.choice(next_functions),
        'unit': random.choice(units),
        'optimizer': random.choice(optimizers),
        'losses': random.choice(dense_losses),
    } for _ in range(population_size)]

    overall_best_model = None
    overall_best_accuracy = 0

    for generation in range(generations):
        print(f"\nGeneration {generation+1}")

        scores = []
        for individual in population:
            model = create_parallel_cnn_model(**individual)
            accuracy = evaluate_model(model, X_train_reshaped, y_train_encoded, X_test_reshaped, y_test_encoded)
            scores.append((individual, accuracy))
            print(f"Evaluated Model: {individual}, Accuracy: {accuracy:.4f}")

            if accuracy > overall_best_accuracy:
                overall_best_model = individual
                overall_best_accuracy = accuracy


        winners = []
        for _ in range(population_size):
            winner = tournament_selection(scores, tournament_size)
            winners.append(winner)


        new_population = []
        for _ in range(0, population_size, 2):
            parent1, parent2 = random.sample(winners, 2)
            child1, child2 = parent1.copy(), parent2.copy()

            if random.random() < 0.5:
                child1['kernel_size'], child2['kernel_size'] = child2['kernel_size'], child1['kernel_size']
                print(f"Crossover happened: {child1['kernel_size']} swapped with {child2['kernel_size']}")

            # Mutation
            if random.random() < 0.1:
                child1['num_filters'] = random.choice(n_filters_options)
                print("Mutation: child1 num_filters changed")
            if random.random() < 0.1:
                child2['pool_size'] = random.choice(pool_sizes_options)
                print("Mutation: child2 pool_size changed")

            new_population.extend([child1, child2])

        population = new_population[:population_size]

    print(f"\nOverall Best Model: {overall_best_model}, Accuracy: {overall_best_accuracy:.4f}")

genetic_algorithm()


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(



Generation 1
Evaluated Model: {'num_filters': 128, 'kernel_size': 5, 'pool_size': 3, 'activation_function': 'elu', 'next_function': 'elu', 'unit': 64, 'optimizer': <class 'keras.src.optimizers.adagrad.Adagrad'>, 'losses': <function mean_absolute_error at 0x78982624b520>}, Accuracy: 0.1880
Evaluated Model: {'num_filters': 32, 'kernel_size': 3, 'pool_size': 3, 'activation_function': 'elu', 'next_function': 'relu', 'unit': 256, 'optimizer': <class 'keras.src.optimizers.adagrad.Adagrad'>, 'losses': <function categorical_crossentropy at 0x789826274550>}, Accuracy: 0.7672
Evaluated Model: {'num_filters': 128, 'kernel_size': 7, 'pool_size': 5, 'activation_function': 'relu', 'next_function': 'elu', 'unit': 256, 'optimizer': <class 'keras.src.optimizers.adagrad.Adagrad'>, 'losses': <function categorical_crossentropy at 0x789826274550>}, Accuracy: 0.6885
Evaluated Model: {'num_filters': 128, 'kernel_size': 5, 'pool_size': 3, 'activation_function': 'elu', 'next_function': 'sigmoid', 'unit': 256,

In [ ]:
import numpy as np
import pandas as pd
import random
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop
from tensorflow.keras.losses import categorical_crossentropy, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder

# Assuming the data loading and preprocessing steps are done
X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None)

X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None)

# Reshape data
X_train_reshaped = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# One-hot encode labels
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Hyperparameters space
n_filters_options = [32, 64, 128]
kernel_sizes_options = [3, 5, 7]
pool_sizes_options = [2, 3, 5]
activation_functions = ['relu', 'sigmoid', 'elu']
next_functions = ['relu', 'sigmoid', 'elu']
units = [64, 128, 256]
optimizers = [Adam, Adagrad, RMSprop]
dense_losses = [categorical_crossentropy, mean_squared_error, mean_absolute_error]
rates = [0.1, 0.01, 0.001]
batch_sizes = [32, 64, 128]
epochs = [10, 20, 30]

from keras.models import Model
from keras.layers import Concatenate, Average, Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam

def create_parallel_cnn_model(num_filters, kernel_size, pool_size, activation_function, next_function, unit, optimizer, losses, l_rate, batch, n_epoch):

    input_shape = (X_train_reshaped.shape[1], 1)
    input_layer = Input(shape=input_shape)

    conv1a = Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function)(input_layer)
    pool1a = MaxPooling1D(pool_size=pool_size)(conv1a)

    conv1b = Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function)(input_layer)
    pool1b = MaxPooling1D(pool_size=pool_size)(conv1b)

    merged = Average()([pool1a, pool1b])

    conv2 = Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function)(merged)
    pool2 = MaxPooling1D(pool_size=pool_size)(conv2)

    flat = Flatten()(pool2)
    dense = Dense(units=unit, activation=next_function)(flat)
    dropout = Dropout(0.5)(dense)
    output_layer = Dense(y_train_encoded.shape[1], activation='softmax')(dropout)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=optimizer(learning_rate=l_rate), loss=losses, metrics=['accuracy'])

    return model

def evaluate_model(model, X_train, y_train, X_test, y_test, batch, n_epoch):
    model.fit(X_train, y_train, epochs=n_epoch, batch_size=batch, verbose=0)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

def tournament_selection(population_scores, tournament_size=3):
    tournament = random.sample(population_scores, tournament_size)
    tournament.sort(key=lambda x: x[1], reverse=True)
    return tournament[0][0]

def genetic_algorithm(generations=5, population_size=25, tournament_size=3):
    population = [{
        'num_filters': random.choice(n_filters_options),
        'kernel_size': random.choice(kernel_sizes_options),
        'pool_size': random.choice(pool_sizes_options),
        'activation_function': random.choice(activation_functions),
        'next_function': random.choice(next_functions),
        'unit': random.choice(units),
        'optimizer': random.choice(optimizers),
        'losses': random.choice(dense_losses),
        'l_rate': random.choice(rates),
        'batch': random.choice(batch_sizes),
        'n_epoch': random.choice(epochs),
    } for _ in range(population_size)]

    overall_best_model = None
    overall_best_accuracy = 0

    for generation in range(generations):
        print(f"\nGeneration {generation+1}")

        scores = []
        for individual in population:
            model = create_parallel_cnn_model(**individual)
            accuracy = evaluate_model(model, X_train_reshaped, y_train_encoded, X_test_reshaped, y_test_encoded, individual['batch'], individual['n_epoch'])
            scores.append((individual, accuracy))
            print(f"Evaluated Model: {individual}, Accuracy: {accuracy:.4f}")

            if accuracy > overall_best_accuracy:
                overall_best_model = individual
                overall_best_accuracy = accuracy

        winners = []
        for _ in range(population_size):
            winner = tournament_selection(scores, tournament_size)
            winners.append(winner)

        new_population = []
        for _ in range(0, population_size, 2):
            parent1, parent2 = random.sample(winners, 2)
            child1, child2 = parent1.copy(), parent2.copy()

            if random.random() < 0.5:
                child1['kernel_size'], child2['kernel_size'] = child2['kernel_size'], child1['kernel_size']
                print(f"Crossover happened: {child1['kernel_size']} swapped with {child2['kernel_size']}")

            # Mutation
            if random.random() < 0.1:
                child1['num_filters'] = random.choice(n_filters_options)
                print("Mutation: child1 num_filters changed")
            if random.random() < 0.1:
                child2['pool_size'] = random.choice(pool_sizes_options)
                print("Mutation: child2 pool_size changed")

            new_population.extend([child1, child2])

        population = new_population[:population_size]

    print(f"\nOverall Best Model: {overall_best_model}, Accuracy: {overall_best_accuracy:.4f}")

genetic_algorithm()


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(



Generation 1
Evaluated Model: {'num_filters': 64, 'kernel_size': 3, 'pool_size': 2, 'activation_function': 'relu', 'next_function': 'relu', 'unit': 64, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'losses': <function mean_absolute_error at 0x7c7f24435750>, 'l_rate': 0.1, 'batch': 64, 'n_epoch': 20}, Accuracy: 0.1683
Evaluated Model: {'num_filters': 32, 'kernel_size': 3, 'pool_size': 3, 'activation_function': 'sigmoid', 'next_function': 'relu', 'unit': 64, 'optimizer': <class 'keras.src.optimizers.rmsprop.RMSprop'>, 'losses': <function mean_squared_error at 0x7c7f24435480>, 'l_rate': 0.01, 'batch': 64, 'n_epoch': 30}, Accuracy: 0.9203
Evaluated Model: {'num_filters': 32, 'kernel_size': 5, 'pool_size': 2, 'activation_function': 'elu', 'next_function': 'sigmoid', 'unit': 128, 'optimizer': <class 'keras.src.optimizers.rmsprop.RMSprop'>, 'losses': <function mean_absolute_error at 0x7c7f24435750>, 'l_rate': 0.1, 'batch': 128, 'n_epoch': 30}, Accuracy: 0.1683
Evaluated Model: {'num

**Testing Outputs of GA in CNN Models**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, Average
from tensorflow.keras.optimizers import Adagrad
from sklearn.preprocessing import OneHotEncoder
import time

start_time = time.time()

X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None, names=['Activity'])['Activity']

X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None, names=['Activity'])['Activity']

X_train_array = X_train.values
X_test_array = X_test.values

X_train_reshaped = X_train_array.reshape((X_train_array.shape[0], X_train_array.shape[1], 1))
X_test_reshaped = X_test_array.reshape((X_test_array.shape[0], X_test_array.shape[1], 1))

encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1))



def create_parallel_cnn_model(n_filters, kernel_size, pool_size, activation_function, learning_rate):

    inputs = Input(shape=(X_train_reshaped.shape[1], 1))


    conv1 = Conv1D(filters=n_filters, kernel_size=kernel_size, activation=activation_function)(inputs)
    pool1 = MaxPooling1D(pool_size=pool_size)(conv1)


    conv2 = Conv1D(filters=n_filters, kernel_size=kernel_size, activation=activation_function)(inputs)
    pool2 = MaxPooling1D(pool_size=pool_size)(conv2)


    merged = Average()([pool1, pool1])

    # Continue with normal flow
    conv3 = Conv1D(filters=n_filters, kernel_size=kernel_size, activation=activation_function)(merged)
    pool3 = MaxPooling1D(pool_size=pool_size)(conv3)
    flat = Flatten()(pool3)
    dense = Dense(128, activation='elu')(flat)
    outputs = Dense(y_train_encoded.shape[1], activation='softmax')(dense)

    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Parameters (You can adjust these based on your needs)
n_filters = 64
kernel_size = 3
pool_size = 3
activation_function = 'elu'  # Add your desired activation function here
learning_rate = 0.01

# Model creation and training
model = create_parallel_cnn_model(n_filters, kernel_size, pool_size, activation_function, learning_rate)
model.fit(X_train_reshaped, y_train_encoded, epochs=30, batch_size=32, verbose=1)

# Evaluation
loss, accuracy = model.evaluate(X_test_reshaped, y_test_encoded, verbose=1)
print(f'Test accuracy: {accuracy}')

end_time = time.time()
total_time = end_time - start_time
print(f"Total runtime of the script: {total_time} seconds")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/30
230/230 [==============================] - 2s 4ms/step - loss: 0.6294 - accuracy: 0.8085
Epoch 2/30
230/230 [==============================] - 1s 4ms/step - loss: 0.1327 - accuracy: 0.9490
Epoch 3/30
230/230 [==============================] - 1s 4ms/step - loss: 0.0877 - accuracy: 0.9665
Epoch 4/30
230/230 [==============================] - 1s 4ms/step - loss: 0.0916 - accuracy: 0.9655
Epoch 5/30
230/230 [==============================] - 1s 4ms/step - loss: 0.5896 - accuracy: 0.9453
Epoch 6/30
230/230 [==============================] - 1s 4ms/step - loss: 3.7520 - accuracy: 0.9074
Epoch 7/30
230/230 [==============================] - 1s 4ms/step - loss: 0.1576 - accuracy: 0.9689
Epoch 8/30
230/230 [==============================] - 1s 4ms/step - loss: 0.1477 - accuracy: 0.9721
Epoch 9/30
230/230 [==============================] - 1s 4ms/step - loss: 0.0559 - accuracy: 0.9845
Epoch 10/30
230/230 [==============================] - 1s 4ms/step - loss: 0.1080 - accuracy: 0.9784

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, Average
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
import time

start_time = time.time()

X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None, names=['Activity'])['Activity']

X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None, names=['Activity'])['Activity']

X_train_array = X_train.values
X_test_array = X_test.values

X_train_reshaped = X_train_array.reshape((X_train_array.shape[0], X_train_array.shape[1], 1))
X_test_reshaped = X_test_array.reshape((X_test_array.shape[0], X_test_array.shape[1], 1))

encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1))



def create_parallel_cnn_model(n_filters, kernel_size, pool_size, activation_function, learning_rate=0.001):

    inputs = Input(shape=(X_train_reshaped.shape[1], 1))


    conv1 = Conv1D(filters=n_filters, kernel_size=kernel_size, activation=activation_function)(inputs)
    pool1 = MaxPooling1D(pool_size=pool_size)(conv1)


    conv2 = Conv1D(filters=n_filters, kernel_size=kernel_size, activation=activation_function)(inputs)
    pool2 = MaxPooling1D(pool_size=pool_size)(conv2)


    merged = Average()([pool1, pool1])

    # Continue with normal flow
    conv3 = Conv1D(filters=n_filters, kernel_size=kernel_size, activation=activation_function)(merged)
    pool3 = MaxPooling1D(pool_size=pool_size)(conv3)
    flat = Flatten()(pool3)
    dense = Dense(128, activation='elu')(flat)
    outputs = Dense(y_train_encoded.shape[1], activation='softmax')(dense)

    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

    return model

# Parameters (You can adjust these based on your needs)
n_filters = 128
kernel_size = 5
pool_size = 3
activation_function = 'relu'  # Add your desired activation function here
learning_rate = 0.001

# Model creation and training
model = create_parallel_cnn_model(n_filters, kernel_size, pool_size, activation_function, learning_rate)
model.fit(X_train_reshaped, y_train_encoded, epochs=10, batch_size=32, verbose=1)

# Evaluation
loss, accuracy = model.evaluate(X_test_reshaped, y_test_encoded, verbose=1)
print(f'Test accuracy: {accuracy}')

end_time = time.time()
total_time = end_time - start_time
print(f"Total runtime of the script: {total_time} seconds")


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


230/230 [==============================] - 3s 6ms/step - loss: 0.0361 - accuracy: 0.8509
Epoch 2/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0098 - accuracy: 0.9619
Epoch 3/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0068 - accuracy: 0.9739
Epoch 4/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0052 - accuracy: 0.9792
Epoch 5/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0046 - accuracy: 0.9826
Epoch 6/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0035 - accuracy: 0.9865
Epoch 7/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0031 - accuracy: 0.9882
Epoch 8/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0028 - accuracy: 0.9886
Epoch 9/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0028 - accuracy: 0.9890
Epoch 10/10
93/93 [==============================] - 0s 4ms/step - loss: 0.0141 - accuracy: 0.9457
Test accurac

In [ ]:
import numpy as np
import pandas as pd
import random
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder

# Assuming the data loading and preprocessing steps are done
X_train = pd.read_csv('/content/drive/MyDrive/UCI_HAR_Dataset/UCI/train/X_train.txt', delim_whitespace=True, header=None)
y_train = pd.read_csv('/content/drive/MyDrive/UCI_HAR_Dataset/UCI/train/y_train.txt', delim_whitespace=True, header=None)

X_test = pd.read_csv('/content/drive/MyDrive/UCI_HAR_Dataset/UCI/test/X_test.txt', delim_whitespace=True, header=None)
y_test = pd.read_csv('/content/drive/MyDrive/UCI_HAR_Dataset/UCI/test/y_test.txt', delim_whitespace=True, header=None)

# Reshape data
X_train_reshaped = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# One-hot encode labels
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Hyperparameters space
n_filters_options = [32, 64, 128]
kernel_sizes_options = [3, 5, 7]
pool_sizes_options = [2, 3, 5]
activation_functions = ['relu', 'sigmoid', 'elu']

def create_cnn_model(num_filters, kernel_size, pool_size, activation_function):
    model = Sequential([
        Input(shape=(X_train_reshaped.shape[1], 1)),
        Conv1D(filters=num_filters, kernel_size=kernel_size, activation=activation_function),
        MaxPooling1D(pool_size=pool_size),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(y_train_encoded.shape[1], activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy
# Tournament Selection Function
def tournament_selection(population_scores, tournament_size=3):
    # Randomly select tournament_size individuals
    tournament = random.sample(population_scores, tournament_size)
    # Sort the selected individuals by their fitness score
    tournament.sort(key=lambda x: x[1], reverse=True)
    # The individual with the highest fitness wins the tournament
    return tournament[0][0]

def genetic_algorithm(generations=5, population_size=10, tournament_size=3):
    population = [{
        'num_filters': random.choice(n_filters_options),
        'kernel_size': random.choice(kernel_sizes_options),
        'pool_size': random.choice(pool_sizes_options),
        'activation_function': random.choice(activation_functions)
    } for _ in range(population_size)]

    for generation in range(generations):
        print(f"\nGeneration {generation+1}")

        # Evaluate the population
        scores = []
        for individual in population:
            model = create_cnn_model(**individual)
            accuracy = evaluate_model(model, X_train_reshaped, y_train_encoded, X_test_reshaped, y_test_encoded)
            scores.append((individual, accuracy))
            print(f"Model with {individual} achieved accuracy: {accuracy:.4f}")

        # Apply tournament selection to pick the next generation
        new_population = []
        for _ in range(population_size):
            winner = tournament_selection(scores, tournament_size)
            new_population.append(winner)

        # Mutate the new population to introduce variability
        for individual in new_population:
            if random.random() < 0.1:
                individual['num_filters'] = random.choice(n_filters_options)
            if random.random() < 0.1:
                individual['kernel_size'] = random.choice(kernel_sizes_options)
            if random.random() < 0.1:
                individual['pool_size'] = random.choice(pool_sizes_options)
            if random.random() < 0.1:
                individual['activation_function'] = random.choice(activation_functions)

        population = new_population

    # Find and print the best model from the last generation
    scores.sort(key=lambda x: x[1], reverse=True)
    best_individual = scores[0][0]
    best_accuracy = scores[0][1]
    print(f"\nOverall Best Model: {best_individual} with accuracy: {best_accuracy:.4f}")

# Function calls and data loading should be added as per your setup
genetic_algorithm()


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(



Generation 1
Model with {'num_filters': 128, 'kernel_size': 7, 'pool_size': 2, 'activation_function': 'sigmoid'} achieved accuracy: 0.1822
Model with {'num_filters': 32, 'kernel_size': 5, 'pool_size': 5, 'activation_function': 'relu'} achieved accuracy: 0.9491
Model with {'num_filters': 32, 'kernel_size': 7, 'pool_size': 5, 'activation_function': 'sigmoid'} achieved accuracy: 0.6084
Model with {'num_filters': 32, 'kernel_size': 5, 'pool_size': 5, 'activation_function': 'elu'} achieved accuracy: 0.9532
Model with {'num_filters': 64, 'kernel_size': 3, 'pool_size': 3, 'activation_function': 'elu'} achieved accuracy: 0.9491
Model with {'num_filters': 128, 'kernel_size': 3, 'pool_size': 5, 'activation_function': 'relu'} achieved accuracy: 0.9328
Model with {'num_filters': 32, 'kernel_size': 3, 'pool_size': 2, 'activation_function': 'sigmoid'} achieved accuracy: 0.1822
Model with {'num_filters': 128, 'kernel_size': 7, 'pool_size': 5, 'activation_function': 'relu'} achieved accuracy: 0.9586


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
import time

start_time = time.time()

X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None, names=['Activity'])['Activity']

X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None, names=['Activity'])['Activity']

X_train_array = X_train.values
X_test_array = X_test.values

X_train_reshaped = X_train_array.reshape((X_train_array.shape[0], X_train_array.shape[1], 1))
X_test_reshaped = X_test_array.reshape((X_test_array.shape[0], X_test_array.shape[1], 1))

encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1))

def create_model(n_filters, kernel_size, pool_size, learning_rate=0.001):
    model = Sequential([
        Conv1D(filters=n_filters, kernel_size=kernel_size, activation='elu', input_shape=(X_train_reshaped.shape[1], 1)),
        MaxPooling1D(pool_size=pool_size),
        Flatten(),
        Dense(100, activation='elu'),
        Dense(y_train_encoded.shape[1], activation='softmax')
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

n_filters = 128
kernel_size = 3
pool_size = 2
learning_rate = 0.001

model = create_model(n_filters, kernel_size, pool_size, learning_rate)

model.fit(X_train_reshaped, y_train_encoded, epochs=10, batch_size=32, verbose=1)

loss, accuracy = model.evaluate(X_test_reshaped, y_test_encoded, verbose=1)
print(f'Test accuracy: {accuracy}')

end_time = time.time()
total_time = end_time - start_time
print(f"Total runtime of the script: {total_time} seconds")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10
230/230 [==============================] - 4s 5ms/step - loss: 0.5358 - accuracy: 0.8331
Epoch 2/10
230/230 [==============================] - 1s 5ms/step - loss: 0.1074 - accuracy: 0.9592
Epoch 3/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0609 - accuracy: 0.9792
Epoch 4/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0461 - accuracy: 0.9820
Epoch 5/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0401 - accuracy: 0.9868
Epoch 6/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0353 - accuracy: 0.9860
Epoch 7/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0345 - accuracy: 0.9874
Epoch 8/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0269 - accuracy: 0.9895
Epoch 9/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0422 - accuracy: 0.9859
Epoch 10/10
93/93 [==============================] - 0s 3ms/step - loss: 0.1831 - accuracy: 0.9437
T

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
import time

start_time = time.time()

X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None, names=['Activity'])['Activity']

X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None, names=['Activity'])['Activity']

X_train_array = X_train.values
X_test_array = X_test.values

X_train_reshaped = X_train_array.reshape((X_train_array.shape[0], X_train_array.shape[1], 1))
X_test_reshaped = X_test_array.reshape((X_test_array.shape[0], X_test_array.shape[1], 1))

encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1))

def create_model(n_filters, kernel_size, pool_size, learning_rate=0.001):
    model = Sequential([
        Conv1D(filters=n_filters, kernel_size=kernel_size, activation='relu', input_shape=(X_train_reshaped.shape[1], 1)),
        MaxPooling1D(pool_size=pool_size),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(y_train_encoded.shape[1], activation='softmax')
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

n_filters = 128
kernel_size = 7
pool_size = 2
learning_rate = 0.001

model = create_model(n_filters, kernel_size, pool_size, learning_rate)

model.fit(X_train_reshaped, y_train_encoded, epochs=10, batch_size=32, verbose=1)

loss, accuracy = model.evaluate(X_test_reshaped, y_test_encoded, verbose=1)
print(f'Test accuracy: {accuracy}')

end_time = time.time()
total_time = end_time - start_time
print(f"Total runtime of the script: {total_time} seconds")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10
230/230 [==============================] - 5s 5ms/step - loss: 0.3091 - accuracy: 0.8779
Epoch 2/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0807 - accuracy: 0.9720
Epoch 3/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0621 - accuracy: 0.9763
Epoch 4/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0488 - accuracy: 0.9829
Epoch 5/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0355 - accuracy: 0.9874
Epoch 6/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0259 - accuracy: 0.9906
Epoch 7/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0263 - accuracy: 0.9912
Epoch 8/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0241 - accuracy: 0.9909
Epoch 9/10
230/230 [==============================] - 1s 5ms/step - loss: 0.0226 - accuracy: 0.9918
Epoch 10/10
93/93 [==============================] - 0s 4ms/step - loss: 0.2562 - accuracy: 0.9315
T

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, Average
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
import time

start_time = time.time()

X_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None, names=['Activity'])['Activity']

X_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt', delim_whitespace=True, header=None, names=['Activity'])['Activity']

X_train_array = X_train.values
X_test_array = X_test.values

X_train_reshaped = X_train_array.reshape((X_train_array.shape[0], X_train_array.shape[1], 1))
X_test_reshaped = X_test_array.reshape((X_test_array.shape[0], X_test_array.shape[1], 1))

encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1))



def create_parallel_cnn_model(n_filters, kernel_size, pool_size, activation_function, learning_rate=0.001):

    inputs = Input(shape=(X_train_reshaped.shape[1], 1))


    conv1 = Conv1D(filters=n_filters, kernel_size=kernel_size, activation=activation_function)(inputs)
    pool1 = MaxPooling1D(pool_size=pool_size)(conv1)


    conv2 = Conv1D(filters=n_filters, kernel_size=kernel_size, activation=activation_function)(inputs)
    pool2 = MaxPooling1D(pool_size=pool_size)(conv2)


    merged = Average()([pool1, pool1])

    # Continue with normal flow
    conv3 = Conv1D(filters=n_filters, kernel_size=kernel_size, activation=activation_function)(merged)
    pool3 = MaxPooling1D(pool_size=pool_size)(conv3)
    flat = Flatten()(pool3)
    dense = Dense(128, activation='relu')(flat)
    outputs = Dense(y_train_encoded.shape[1], activation='softmax')(dense)

    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Parameters (You can adjust these based on your needs)
n_filters = 128
kernel_size = 7
pool_size = 2
activation_function = 'relu'  # Add your desired activation function here
learning_rate = 0.001

# Model creation and training
model = create_parallel_cnn_model(n_filters, kernel_size, pool_size, activation_function, learning_rate)
model.fit(X_train_reshaped, y_train_encoded, epochs=10, batch_size=32, verbose=1)

# Evaluation
loss, accuracy = model.evaluate(X_test_reshaped, y_test_encoded, verbose=1)
print(f'Test accuracy: {accuracy}')

end_time = time.time()
total_time = end_time - start_time
print(f"Total runtime of the script: {total_time} seconds")


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


230/230 [==============================] - 3s 7ms/step - loss: 0.3569 - accuracy: 0.8589
Epoch 2/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0924 - accuracy: 0.9671
Epoch 3/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0570 - accuracy: 0.9786
Epoch 4/10
230/230 [==============================] - 2s 7ms/step - loss: 0.0370 - accuracy: 0.9867
Epoch 5/10
230/230 [==============================] - 2s 7ms/step - loss: 0.0294 - accuracy: 0.9903
Epoch 6/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0234 - accuracy: 0.9921
Epoch 7/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0231 - accuracy: 0.9922
Epoch 8/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0350 - accuracy: 0.9872
Epoch 9/10
230/230 [==============================] - 1s 6ms/step - loss: 0.0269 - accuracy: 0.9908
Epoch 10/10
93/93 [==============================] - 0s 3ms/step - loss: 0.1518 - accuracy: 0.9559
Test accurac